In [14]:
import numpy as np
from itertools import combinations
from numpy import random

In [15]:
# return a list of all size k subset of [n]
def get_coord(n, k):
    temp = [i +1 for i in range(n)]
    return list(combinations(temp, k))

# generate cov matrix 
# n is number of vertices 
# vals[i] is the covariance of S, S' when intersection is of size i (e.g. we should have vals 0, 1, 2, 3)
def cov_mat(n, k, vals):
    coordinate = get_coord(n, k)
    l_c = len(coordinate)
    cov_mat = np.zeros((l_c, l_c))
    for i in range(l_c):
        cov_mat[i][i] = vals[k]
        for j in range(i):
            size_inter = len(set(coordinate[i]).intersection(coordinate[j]))
            cov_mat[i][j] = vals[size_inter]
            cov_mat[j][i] = vals[size_inter]
    return cov_mat
    
def check_psd(M):
    try:
        np.linalg.cholesky(M)
        return True
    except np.linalg.LinAlgError:
            return False

def sample_gaussain(mean, M):
    return random.multivariate_normal(mean, M)

In [12]:
# set parameter

n = 10
k = 3
p = .5

# get list of all size k subsets
V = get_coord(n, k)

# writedown list of moments vals[i] is E[S,S'] when |S \cap S'| = i
vals = np.array([p**6, p**6, p**5, p**3])
means = vals[-1] * np.ones(len(V))
# get covariance
vals -= vals[-1] ** 2

cov = cov_mat(n, k, vals)
# check covariance is psd
check_psd(cov)

True

In [11]:
# test the covariance matrix put right values based on size of intersection
def test_cov(M, V, vals, i, j):
    print(V[i], V[j])
    size_int = len([a for a in V[i] if a in V[j]])
    print(M[i][j] == vals[size_int])

In [9]:
test_cov(cov, V, vals, 0, 1)

(1, 2, 3) (1, 2, 4)
True


In [18]:
samples = sample_gaussain(means, cov)